In [105]:
#This is the notebook to set and train the forecaster
PATH_TO_PROFILES = ''
PATH_TO_DATA = ''

In [106]:
import sys
sys.path.insert(0,"/Users/augustecousin/Documents/bel_gm_tool/gm_tool/")

In [107]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor

from datamanager.DM_Belgique import DM_Belgique

In [108]:
data_manager = DM_Belgique()
#path = '../../../../Pivot & Co/Bel 2022 - 1. Growth Modelling/2. Design (Tool)/_Raoul/220207_Belgium_Dataset.csv'
data_manager.open_excel()
data_manager.df.rename(columns={'ACV': 'Distribution'}, inplace=True)


In [109]:
df = data_manager.df.copy()
float_cols = ['Sales in value','Sales in volume', 'Distribution', 'Price per Volume',
              'Price without Promo', 'Sales value with Promo', 'Sales volume with Promo']
df[float_cols] = df[float_cols].apply(pd.to_numeric, errors = 'coerce', axis=1)

In [110]:
categories = df["Category"].unique().tolist()

sub_categories = df["Sub Category"].unique().tolist()
sub_categories.remove("ALL SUB CATEGORIES")

In [111]:
brands_to_remove = ["ALL BRANDS", "PL", "ZZ OTHERS", "SS.MARQUE", "UNBRANDED"]
brands_to_keep = [brand for brand in df["Brand"].unique() if brand not in brands_to_remove]


In [112]:
dict_dates = {"M1": {"Min": "2018-04-01", "Max": "2018-05-01"}, 
              "M2": {"Min": "2018-05-01", "Max": "2018-06-01"}, 
              "M3": {"Min": "2018-06-01", "Max": "2018-07-01"}, 
              "M4": {"Min": "2018-07-01", "Max": "2018-08-01"}, 
              "M5": {"Min": "2018-08-01", "Max": "2018-09-01"}, 
              "M6": {"Min": "2018-09-01", "Max": "2018-10-01"}, 
              "M7": {"Min": "2018-10-01", "Max": "2018-11-01"}, 
              "M8": {"Min": "2018-11-01", "Max": "2018-12-01"}, 
              "M9": {"Min": "2018-12-01", "Max": "2019-01-01"}, 
              "M10": {"Min": "2019-01-01", "Max": "2019-02-01"}, 
              "M11": {"Min": "2019-02-01", "Max": "2019-03-01"}, 
              "M12": {"Min": "2019-03-01", "Max": "2019-04-01"}, 
              "M13": {"Min": "2019-04-01", "Max": "2019-05-01"}, 
              "M14": {"Min": "2019-05-01", "Max": "2019-06-01"}, 
              "M15": {"Min": "2019-06-01", "Max": "2019-07-01"}, 
              "M16": {"Min": "2019-07-01", "Max": "2019-08-01"}, 
              "M17": {"Min": "2019-08-01", "Max": "2019-09-01"}, 
              "M18": {"Min": "2019-09-01", "Max": "2019-10-01"}, 
              "M19": {"Min": "2019-10-01", "Max": "2019-11-01"}, 
              "M20": {"Min": "2019-11-01", "Max": "2019-12-01"}, 
              "M21": {"Min": "2019-12-01", "Max": "2020-01-01"}, 
              "M22": {"Min": "2020-01-01", "Max": "2020-02-01"}, 
              "M23": {"Min": "2020-02-01", "Max": "2020-03-01"}, 
              "M24": {"Min": "2020-03-01", "Max": "2020-04-01"}, 
              "M25": {"Min": "2020-04-01", "Max": "2020-05-01"}, 
              "M26": {"Min": "2020-05-01", "Max": "2020-06-01"}, 
              "M27": {"Min": "2020-06-01", "Max": "2020-07-01"}, 
              "M28": {"Min": "2020-07-01", "Max": "2020-08-01"}, 
              "M29": {"Min": "2020-08-01", "Max": "2020-09-01"}, 
              "M30": {"Min": "2020-09-01", "Max": "2020-10-01"}, 
              "M31": {"Min": "2020-10-01", "Max": "2020-11-01"}, 
              "M32": {"Min": "2020-11-01", "Max": "2020-12-01"}, 
              "M33": {"Min": "2020-12-01", "Max": "2021-01-01"}, 
              "M34": {"Min": "2021-01-01", "Max": "2021-02-01"}, 
              "M35": {"Min": "2021-02-01", "Max": "2021-03-01"}, 
              "M36": {"Min": "2021-03-01", "Max": "2021-04-01"}, 
              "Y1": {"Min": "2018-04-01", "Max": "2019-04-01"}, 
              "Y2": {"Min": "2019-04-01", "Max": "2020-04-01"}, 
              "Y3": {"Min": "2020-04-01", "Max": "2021-04-01"}, 
              "3Y": {"Min": "2018-04-01", "Max": "2021-04-01"}}

In [113]:
def get_data(df, category="all", subcategory="all", brand="all", 
             date_min="all", date_max="all", period_min="all", period_max="all"):
    
    if category != "all":
        df = df[df["Category"].isin(category)]
    if subcategory != "all":
        df = df[df["Sub Category"].isin(subcategory)]
    if brand != "all":
        df = df[df["Brand"].isin(brand)]
    if date_min != "all":
        df = df[df["Date"] >= date_min]
    if date_max != "all":
        df = df[df["Date"] < date_max]
    if period_min != "all":
        df = df[df["Period"] >= period_min]
    if period_max != "all":
        df = df[df["Period"] <= period_max]
    
    return df

In [114]:
df_eda = get_data(df=df, 
         category="all", brand=[brand for brand in brands_to_keep], 
         period_min=1, period_max=40)[["Date", "Period", "Category", "Sub Category", "Brand", "Distribution", "Price per Volume", "Price without Promo", "Sales in volume"]]

df_eda = df_eda.reset_index(drop=True)

In [115]:
df_brand_categ = df.groupby(['Brand'])['Category'].agg(Number_of_Categories = "nunique").reset_index().rename(columns={'Number_of_Categories':'Number of Categories'})

In [116]:
df_eda = pd.merge(df_eda, df_brand_categ, on="Brand", how="left")

In [117]:
df_brand_sub_cat = df.groupby(['Brand', 'Category'])['Sub Category'].agg(Sub_Categories='nunique').reset_index().rename(columns={'Sub_Categories':'Number of Sub Categories'})

In [118]:
df_eda = pd.merge(df_eda, df_brand_sub_cat, on=["Brand", 'Category'], how="left")

In [119]:
df_eda.insert(2, "Week of year", df_eda.Date.dt.weekofyear.values.tolist())
df_eda.insert(3, "Is month start", df_eda.Date.dt.is_month_start.astype(int).values.tolist())
df_eda.insert(3, "Is month end", df_eda.Date.dt.is_month_end.astype(int).values.tolist())

/var/folders/3n/8sjtl4yd5k518vq7tdlbr_nm0000gn/T/ipykernel_39737/267384797.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_eda.insert(2, "Week of year", df_eda.Date.dt.weekofyear.values.tolist())


In [120]:
#train = df_train_test_lag_roll_ewm_ohe.loc[(df_train_test_lag_roll_ewm_ohe["Period"] < 37), :]
#val = df_train_test_lag_roll_ewm_ohe.loc[(df_train_test_lag_roll_ewm_ohe["Period"] >= 37), :]

In [121]:
def random_noise(df):
    return np.random.normal(scale=0.01, size=(len(df),))

def lag_features(df, lags):
    for lag in lags:
        df['Sales lag ' + str(lag)] = df.groupby(["Category", "Sub Category", "Brand"])['Sales in volume'].transform(
            lambda x: x.shift(lag)) + random_noise(df)
        df['Sales lag ' + str(lag)] = df['Sales lag ' + str(lag)].apply(lambda x: 0 if x < 0 else x)
    return df

def roll_mean_features(df, windows):
    for window in windows:
        df['Sales roll mean ' + str(window)] = df.groupby(["Category", "Sub Category", "Brand"])['Sales in volume'].transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=2, win_type="triang").mean()) + random_noise(df)
    return df

def ewm_features(df, alphas, lags):
    for alpha in alphas:
        for lag in lags:
            df['Sales ewm alpha ' + str(alpha).replace(".", "") + " lag " + str(lag)] = \
                df.groupby(["Category", "Sub Category", "Brand"])['Sales in volume'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return df

In [127]:
lags_list = [1, 2, 3, 4, 5, 6, 9, 13, 15, 20, 26] 
df_train_test_lag = lag_features(df_eda, lags_list)

In [129]:
roll_mean_list = [2, 3, 6, 9, 13, 20, 26, 30, 39] 
df_train_test_lag_roll = roll_mean_features(df_train_test_lag, roll_mean_list)

In [132]:
alphas_list = [0.95, 0.9, 0.8, 0.7, 0.5]
lags_list = [1, 2, 3, 4, 5, 6, 9, 13, 15, 20, 26]
df_train_test_lag_roll_ewm = ewm_features(df_train_test_lag_roll, alphas_list, lags_list)

In [136]:
df_train_test_lag_roll_ewm['Sales in volume']

0        1.3403
1        0.5305
2        0.0133
3        0.0008
4        0.0003
          ...  
17155    0.0000
17156    0.0000
17157    0.0000
17158    0.0000
17159    0.0000
Name: Sales in volume, Length: 17160, dtype: float64